In [ ]:
import numpy as np
import pandas as pd
import igraph as ig
import dowhy as dw
import matplotlib.pyplot as plt
import automates.model_analysis.identification_algorithms as ia

# DoWhy Confounding Example

## Objectives
There are two goals:
1. Illustrate that estimating an unidentifiable effect leads to inaccurate estimation
2. Illustrate that ignoring known confounding rather than correcting for it leads to incorrect decisions

I will construct examples that highlight why, under certain conditions, ignoring information about identifiability gained from graph structure can lead to incorrect conclusions.

## Example 1: The Bow-Arc Graph
The bow-arc graph is the smallest unidentifiable graph (Shiptser and Pearl, page 1949), so let us examine this model first.

Suppose a dice is rolled and the outcome $D$ is observed. A second dice is rolled whose outcome $U$ is not observed. The outcome $y_i = 6u_i + 0.75d_i + \mathcal{N}(2, 1)$.

The Structural Causal Model (SCM) has the following structural equations:

\begin{align*} 
U &= f_U(\varepsilon_U) \\
D &= f_D(\varepsilon_H) \\
Y &= f_Y(U, D, \varepsilon_Y), \text{ where } \varepsilon_U \perp \varepsilon_D \perp \varepsilon_Y
\end{align*}

The graph corresponding to the scenario:

In [ ]:
# Specify iGraph
g1 = ig.Graph(edges=[[0, 1], [2, 1], [2, 0]], directed=True)
g1.vs["name"] = ["D", "Y", "U"]; g1.vs["description"] = ["O", "O", "U"]
g1.es["description"] = ["O", "U", "U"]

# plot graph
v_style1 = {"vertex_label": g1.vs["name"], "vertex_size": 20, "layout": g1.layout("rt", mode="all", root=[2])}
fig, ax = plt.subplots(); plt.axis('off')
ig.plot(g1, target=ax, **v_style1)

### Simulation
I sample from the following error distributions:
\begin{align*}
\varepsilon_U &\sim \text{Uniform}(1, 6) \\
\varepsilon_D &\sim \text{Uniform}(1, 6) \\
\varepsilon_Y &\sim \mathcal{N}(2, 1) \\
\end{align*}

Then construct the observations and outcome using the following structural equations for the graph and our known relationship:
\begin{align*}
u_i &= \varepsilon_{u_i} \\
d_i &= \varepsilon_{d_i} \\
y_i &= 6*u_i + 0.75*d_i+\varepsilon_{y_i} \\
\end{align*}

This is repeated 100 times:

In [ ]:
# Sample n times from Error Distributions
n = 20; np.random.seed(54321)
epsilon_U = np.random.randint(1, 7, n)
epsilon_D = np.random.randint(1, 7, n)
epsilon_Y = np.random.normal(2, 1, n)


# Construct Observations and Outcomes
sim_data = pd.DataFrame(np.transpose(np.array([epsilon_D, 6*epsilon_U+0.75*epsilon_D+epsilon_Y])), columns = ["D", "Y"])
sim_data[0:10]

### Identification, ID-Algorithm
Suppose we implement *reference* ID-Algorithm.

In [ ]:
# Implement ID Algorithm on the Graph
results = ia.identifiability(["Y"], ["D"], g1)

We see that the effect $P(Y|do(d))$ is not identifiable by the ID-Algorithm.

### Identification, DoWhy
Now, using DoWhy, I input the graph and data (DoWhy requires that data be bundled with the graph) and then run DoWhy's identification step.

In [ ]:
# Specify DoWhy model
dw_graph = 'digraph {U [observed="no"]; D [observed="yes"]; Y [observed="yes"]; U -> Y; U -> D; D -> Y}'
dw_model = dw.CausalModel(data=sim_data, treatment=["D"], outcome=["Y"], graph=dw_graph)

# Implement DoWhy identification
dw_id = dw_model.identify_effect(estimand_type="nonparametric-ate", method_name="default", proceed_when_unidentifiable=True)
print(dw_id)

We see that DoWhy warns us about the assumptions it's making, but proceeds anyway. It returns a regression coefficient as the "identified" value.

### Estimation, DoWhy
If we ignore DoWhy's warning and proceed with estimation (using linear regression) anyway, we get the following

In [ ]:
# Implement DoWhy linear estimation method
dw_estimate = dw_model.estimate_effect(dw_id, method_name="backdoor.linear_regression",test_significance=True)
print(dw_estimate)

# return output of interest

From the equations we used to simulate the data, we know that the true effect size should be 0.75, which is quite far from what we estimate.

Below is an example of what we could estimate given the values of U as well.

In [ ]:
sim_true = pd.DataFrame(np.transpose(np.array([epsilon_U, epsilon_D, 6*epsilon_U+0.75*epsilon_D+epsilon_Y])), columns = ["U", "D", "Y"])
true_graph = 'digraph {U; D; Y; U -> Y; U -> D; D -> Y}'
true_model = dw.CausalModel(data=sim_true, treatment=["D"], outcome=["Y"], graph=true_graph)
true_id = true_model.identify_effect(estimand_type="nonparametric-ate", method_name="default", proceed_when_unidentifiable=True)
true_estimate = true_model.estimate_effect(true_id, method_name="backdoor.linear_regression",test_significance=True)
print(true_estimate)